In [1]:
import os
import re
import sys
import html
import json
import time
import requests
import subprocess
from datetime import datetime, timedelta, timezone

#默认参数
default_config = {
    "retry_count": 3,
    "url": "http://127.0.0.1:8000",
    "title": "YouTube",
    "link": "https://m.youtube.com",
    "description": "在YouTube 上畅享您喜爱的视频和音乐，上传原创内容并与亲朋好友和全世界观众分享您的视频。",
    "icon": "https://static.vecteezy.com/system/resources/previews/003/399/771/original/youtube-icon-editorial-free-vector.jpg",
    "category": "TV &amp; Film",
    "channelid_youtube": {
        "youtube": {
            "update_size": 15,
            "id": "UCBR8-60-B28hp2BmDPdntcQ",
            "title": "YouTube",
            "quality": "480",
            "last_size": 50,
            "media": "m4a"
        }
    }
}

In [2]:
#日志模块
def write_log(log):
    # 获取当前的具体时间
    current_time = datetime.now()
    # 格式化输出, 只保留年月日时分秒
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    # 打开文件, 并读取原有内容
    try:
        with open("log.txt", "r") as file:
            contents = file.read()
    except FileNotFoundError:
        contents = ""
    # 将新的日志内容添加在原有内容之前
    new_contents = f"{formatted_time} {log}\n{contents}"
    # 将新的日志内容写入文件
    with open("log.txt", "w") as file:
        file.write(new_contents)
    formatted_time_mini = current_time.strftime("%H:%M:%S")
    print(f"{formatted_time_mini}|{log}")

In [3]:
# 安装库模块
def library_install(library):
    # 检查库是否已安装
    def is_library_installed():
        try:
            result = subprocess.run([library , '--version'], capture_output=True, text=True)
            return result.returncode == 0
        except FileNotFoundError:
            return False
    # 如果库未安装, 则尝试安装
    def install_library():
        try:
            result = subprocess.run(['pip', 'install', library , '-U'], capture_output=True, text=True)
            return result.returncode == 0
        except FileNotFoundError:
            return False
    # 如果库已安装, 则尝试更新
    def update_library():
        try:
            result = subprocess.run(['pip', 'install', '--upgrade', library], capture_output=True, text=True)
            return result.returncode == 0
        except FileNotFoundError:
            return False
    # 检查库是否已安装
    if is_library_installed():
        write_log(f"{library}已安装")
    else:
        write_log(f"{library}未安装")
    # 如果库已安装, 则尝试更新
    if is_library_installed():
        if update_library():
            write_log(f"{library}更新成功")
        else:
            write_log(f"{library}更新失败")
    else:  # 如果库未安装, 则尝试安装
        if install_library():
            write_log(f"{library}安装成功")
        else:
            write_log(f"{library}安装失败")

In [4]:
# 安装/更新yt-dlp
library_install("yt-dlp")

00:30:50|yt-dlp已安装
00:30:53|yt-dlp更新成功


In [5]:
import yt_dlp

# 获取视频时长模块
def video_duration(video_website, video_url):
    try:
        # 初始化 yt_dlp 实例, 并忽略错误
        ydl = yt_dlp.YoutubeDL({'ignoreerrors': True})
        # 使用提供的 URL 提取视频信息
        if info_dict := ydl.extract_info(
            f"{video_website}{video_url}", download=False
        ):
            # 获取视频时长并返回
            return info_dict.get('duration')
    except Exception as e:
        # 记录下载失败及错误详情
        write_log((f"{video_url} 下载失败, 错误信息：{str(e)}").replace("ERROR: ", "").replace(f"{video_url}: ", ""))
        return None

# 获取已下载视频时长模块
def get_duration_ffprobe(file_path):
    try:
        # 调用 ffprobe 命令获取视频文件的时长信息
        command = [
            "ffprobe",                       # ffprobe 命令
            "-i", file_path,                 # 输入文件路径
            "-show_entries", "format=duration",  # 显示时长信息
            "-v", "error",
            "-of", "default=noprint_wrappers=1:nokey=1"
        ]
        # 执行命令并获取输出
        output = subprocess.check_output(command, stderr=subprocess.STDOUT).decode("utf-8")
        # 使用正则表达式提取时长值, 并将其转换为浮点数并四舍五入为整数
        return round(float(output))
    except subprocess.CalledProcessError as e:
        write_log(f"Error: {e.output}")
        return None

# 下载视频模块
def download_video(video_url, output_dir, output_format, video_website, format_code=480):
    if output_format == 'm4a':
        format_out = "bestaudio[ext=m4a]/best"   # 音频编码
    else:
        format_out = f'bestvideo[ext=mp4][height<={format_code}]+bestaudio[ext=m4a]/best'  # 视频编码
    ydl_opts = {
        'outtmpl': f'{output_dir}/{video_url}.{output_format}',  # 输出文件路径和名称
        'format': f'{format_out}'  # 指定下载的最佳音频和视频格式
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([f'{video_website}{video_url}'])  # 下载指定视频链接的视频
        write_log(f"{video_url}下载成功")  # 写入下载成功的日志信息
    except Exception as e:
        write_log((f"{video_url}下载失败, 错误信息：{str(e)}").replace("ERROR: ", "").replace(f"{video_url}: ", ""))  # 写入下载失败的日志信息
        return video_url

In [6]:
# 视频完整下载模块
def dl_full_video(video_url, output_dir, output_format, video_website, format_code=480):
    if not (id_duration := video_duration(video_website, video_url)):  # 获取视频时长
        return video_url
    if download_video(video_url, output_dir, output_format, video_website, format_code):  # 下载视频
        return video_url
    duration_video = get_duration_ffprobe(f"{output_dir}/{video_url}.{output_format}")  # 获取已下载视频的实际时长
    if id_duration == duration_video:  # 检查实际时长与预计时长是否一致
        return None
    if duration_video:
        os.remove(f"{output_dir}/{video_url}.{output_format}")  #删除不完整的视频
    return video_url

# 视频重试下载模块
def dl_retry_video(video_url, output_dir, output_format, retry_count, video_website, format_code=480):
    yt_id_failed = dl_full_video(video_url, output_dir, output_format, video_website, format_code)
    # 下载失败后重复尝试下载视频
    yt_id_count = 0
    while yt_id_count < retry_count and yt_id_failed:
        yt_id_count += 1
        write_log(f"{video_url}第{yt_id_count}次重新下载")
        yt_id_failed = dl_full_video(video_url, output_dir, output_format, video_website, format_code)
    return yt_id_failed

In [7]:
# 检查当前文件夹中是否存在config.json文件
if not os.path.exists('config.json'):
    # 如果文件不存在, 创建并写入默认字典
    with open('config.json', 'w') as file:
        json.dump(default_config, file, indent=4)
    write_log("不存在配置文件, 已新建, 默认频道")
    config = default_config
else:
    # 如果文件存在, 读取字典并保存到config变量中
    try:
        with open('config.json', 'r') as file:
            config = json.load(file)
        write_log("已读取配置文件")
    # 如果config格式有问题, 停止运行并报错
    except Exception as e:
        write_log(f"配置文件有误, 请检查config.json, {str(e)}")
        sys.exit(0)

00:30:53|已读取配置文件


In [8]:
# 对retry_count进行纠正
if (
    'retry_count' not in config
    or not isinstance(config['retry_count'], int)
    or config['retry_count'] <= 0
):
    config['retry_count'] = default_config["retry_count"]
# 对url进行纠正
if (
    'url' not in config
    or not re.search(r"^(https?|ftp)://[^\s/$.?#].[^\s]*$", config['url'])
    ):
    config['url'] = default_config["url"]
# 对title进行纠正
if ('title' not in config):
    config['title'] = default_config["title"]
# 对link进行纠正
if (
    'link' not in config
    or not re.search(r"^(https?|ftp)://[^\s/$.?#].[^\s]*$", config['link'])
    ):
    config['link'] = default_config["link"]
# 对description进行纠正
if ('description' not in config):
    config['description'] = default_config["description"]
# 对icon进行纠正
if (
    'icon' not in config
    or not re.search(r"^(https?|ftp)://[^\s/$.?#].[^\s]*$", config['icon'])
    ):
    config['icon'] = default_config["icon"]
# 对category进行纠正
if ('category' not in config):
    config['category'] = default_config["category"]

In [9]:
# 从配置文件中获取YouTube的频道
if 'channelid_youtube' in config:
    channelid_youtube = config["channelid_youtube"]
    write_log("已读取youtube频道信息")
else:
    channelid_youtube = None
    write_log("youtube频道信息不存在")
# 从配置文件中获取bilibili的频道
if 'channelid_bilibili' in config:
    channelid_bilibili = config["channelid_bilibili"]
    write_log("已读取bilibili频道信息")
else:
    channelid_bilibili = None
    write_log("bilibili频道信息不存在")

00:30:53|已读取youtube频道信息
00:30:53|bilibili频道信息不存在


In [10]:
# 构建文件夹channel_id
folder_path_channel_ids = os.path.join(os.getcwd(), "channel_id")
if not os.path.exists(folder_path_channel_ids):  # 判断文件夹是否存在
    os.makedirs(folder_path_channel_ids)  # 创建文件夹
    write_log("文件夹channel_id创建成功")

In [11]:
# 视频分辨率变量
youtube_video_media = ["m4v", "mov", "qt", "avi", "flv", "wmv", "asf", "mpeg", "mpg", "vob", "mkv", "rm", "rmvb", "vob", "ts", "dat"]
youtube_dpi = ["144", "180", "216", "240", "360", "480", "720", "1080", "1440", "2160", "4320"]
youtube_media = ["m4a", "mp4"]
# 复制字典youtube-channelid, 遍历复制后的字典进行操作以避免在循环中删除元素导致的迭代错误
channelid_youtube_copy = channelid_youtube.copy()
# 对youtube-channelid的错误进行更正
for channelid_youtube_key, channelid_youtube_value in channelid_youtube_copy.items():
    # 判断id是否正确
    if 'id' not in channelid_youtube_value or not re.search(r"UC.{22}", channelid_youtube_value['id']):
        # 删除错误的
        del channelid_youtube[channelid_youtube_key]
        write_log(f"YouTube频道 {channelid_youtube_key} ID不正确")
    else:
        # 对update_size进行纠正
        if (
            'update_size' not in channelid_youtube_value
            or not isinstance(channelid_youtube_value['update_size'], int)
            or channelid_youtube_value['update_size'] <= 0
        ):
            channelid_youtube[channelid_youtube_key]['update_size'] = default_config["channelid_youtube"]["youtube"]["update_size"]
        # 对id进行纠正
        channelid_youtube[channelid_youtube_key]['id'] = re.search(r"UC.{22}", channelid_youtube_value['id']).group()
        # 对last_size进行纠正
        if (
            'last_size' not in channelid_youtube_value
            or not isinstance(channelid_youtube_value['last_size'], int)
            or channelid_youtube_value['last_size'] <= 0
        ):
            channelid_youtube[channelid_youtube_key]['last_size'] = default_config["channelid_youtube"]["youtube"]["last_size"]
        # 对title进行纠正
        if 'title' not in channelid_youtube_value:
            channelid_youtube[channelid_youtube_key]['title'] = channelid_youtube_key
        # 对quality进行纠正
        if (
            (
                'quality' not in channelid_youtube_value
                or channelid_youtube_value['quality'] not in youtube_dpi
            )
            and 'media' in channelid_youtube_value
            and channelid_youtube_value['media'] == "mp4"
        ):
            channelid_youtube[channelid_youtube_key]['quality'] = default_config["channelid_youtube"]["youtube"]["quality"]
        # 对media进行纠正
        if (
            'media' in channelid_youtube_value
            and channelid_youtube_value['media'] not in youtube_media
            and channelid_youtube_value['media'] in youtube_video_media
        ):
            channelid_youtube[channelid_youtube_key]['media'] = "mp4"
        elif (
            'media' in channelid_youtube_value
            and channelid_youtube_value['media'] not in youtube_media
            or 'media' not in channelid_youtube_value
        ):
            channelid_youtube[channelid_youtube_key]['media'] = "m4a"

In [12]:
# 读取youtube频道的id
if channelid_youtube is not None:
    channelid_youtube_ids = dict({channel["id"]: key for key, channel in channelid_youtube.items()})
    write_log("读取youtube频道的channelid成功")
else:
    channelid_youtube_ids = None
# 读取bilibili频道的id
if channelid_bilibili is not None:
    channelid_bilibili_ids = [channelid_bilibili[key]['id'] for key in channelid_bilibili]
    write_log("读取bilibili频道的channelid成功")
else:
    channelid_bilibili_ids = None

00:30:53|读取youtube频道的channelid成功


In [13]:
# 更新Youtube频道xml
channelid_youtube_ids_update = {}  #创建需更新的频道
youtube_content_ytid_update = {}  #创建需下载视频列表
# 判断频道id是否正确
pattern_youtube404 = r"Error 404"  # 设置要匹配的正则表达式模式
pattern_youtube_vary = r'([0-9]{4}-[0-9]{2}-[0-9]{2}T[0-2][0-9]:[0-6][0-9]:[0-6][0-9]\+00:00)?(starRating count="[0-9]*")?(statistics views="[0-9]*")?(<id>yt:channel:(UC.{22})?</id>)?(<yt:channelId>(UC.{22})?</yt:channelId>)?'
for youtube_key, youtube_value in channelid_youtube_ids.items():
    # 构建 URL
    youtube_url = f"https://www.youtube.com/feeds/videos.xml?channel_id={youtube_key}"
    # 发送请求并获取响应内容
    youtube_response = requests.get(youtube_url)
    youtube_content = youtube_response.text
    if not re.search(pattern_youtube404, youtube_content):
        youtube_content_clean = re.sub(pattern_youtube_vary, '', youtube_content)
        # 读取原Youtube频道xml文件并判断是否要更新
        try:
            with open(f"channel_id/{youtube_key}.txt", 'r', encoding='utf-8') as file:  # 打开文件进行读取
                youtube_content_original = file.read()  # 读取文件内容
                youtube_content_original_clean = re.sub(pattern_youtube_vary, '', youtube_content_original)
            if youtube_content_clean != youtube_content_original_clean :  #判断是否要更新
                channelid_youtube_ids_update[youtube_key] = youtube_value
        except FileNotFoundError:  #文件不存在直接更新
            channelid_youtube_ids_update[youtube_key] = youtube_value
        # 构建文件路径
        file_path = os.path.join(folder_path_channel_ids, f"{youtube_key}.txt")
        # 构建文件路径
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(youtube_content)
            write_log(f"YouTube频道 {youtube_value} 已更新")
        # 构建频道文件夹
        folder_path_channel_id = os.path.join(os.getcwd(), youtube_key)
        if not os.path.exists(folder_path_channel_id):  # 判断文件夹是否存在
            os.makedirs(folder_path_channel_id)  # 创建文件夹
            write_log(f"文件夹{youtube_key}创建成功")
        #获取Youtube视频ID列表
        youtube_content_ytid = re.findall(r"(?<=<id>yt:video:).{11}(?=</id>)", youtube_content)
        youtube_content_ytid = youtube_content_ytid[:channelid_youtube[youtube_value]['update_size']]
        #获取已下载媒体名称
        youtube_media = ("m4a", "mp4") if channelid_youtube[youtube_value]['media'] == "m4a" else ("mp4")
        youtube_content_ytid_original = [os.path.splitext(file)[0] for file in os.listdir(youtube_key) if file.endswith(youtube_media)]
        if youtube_content_ytid := [
            exclude
            for exclude in youtube_content_ytid
            if exclude not in youtube_content_ytid_original
        ]:
            channelid_youtube_ids_update[youtube_key] = youtube_value
            youtube_content_ytid_update[youtube_key] = youtube_content_ytid
    else:
        write_log(f"YouTube频道 {youtube_value} ID不正确无法获取")
        del channelid_youtube_ids[youtube_key]  # 删除错误ID
if channelid_youtube_ids_update:
    write_log(f"需更新的YouTube频道:{', '.join(channelid_youtube_ids_update.values())}")

00:30:54|YouTube频道 youtube 已更新
00:30:54|YouTube频道 stone记 已更新
00:30:54|YouTube频道 四季妈妈—小龙 已更新
00:30:54|需更新的YouTube频道:youtube, stone记, 四季妈妈—小龙


In [14]:
# 下载YouTube视频
yt_id_failed = []
for ytid_key, ytid_value in youtube_content_ytid_update.items():
    # 获取对应文件类型
    yt_id_file = channelid_youtube[channelid_youtube_ids_update[ytid_key]]['media']
    # 如果为视频格式获取分辨率
    if yt_id_file == "mp4":
        yt_id_quality = channelid_youtube[channelid_youtube_ids_update[ytid_key]]['quality']
    else:
        yt_id_quality = None
    # 下载视频
    for yt_id in ytid_value:
        if dl_retry_video(
            yt_id, ytid_key, yt_id_file, config['retry_count'], "https://www.youtube.com/watch?v=", yt_id_quality
        ):
            yt_id_failed.append(yt_id)
            write_log(f"{yt_id}无法下载")

[youtube] Extracting URL: https://www.youtube.com/watch?v=DcNI16MEjrM
[youtube] DcNI16MEjrM: Downloading webpage
[youtube] DcNI16MEjrM: Downloading ios player API JSON
[youtube] DcNI16MEjrM: Downloading android player API JSON
[youtube] DcNI16MEjrM: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=DcNI16MEjrM
[youtube] DcNI16MEjrM: Downloading webpage
[youtube] DcNI16MEjrM: Downloading ios player API JSON
[youtube] DcNI16MEjrM: Downloading android player API JSON
[youtube] DcNI16MEjrM: Downloading m3u8 information
[info] DcNI16MEjrM: Downloading 1 format(s): 604+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 8
[download] Destination: UCBR8-60-B28hp2BmDPdntcQ/DcNI16MEjrM.f604.mp4
[download] 100% of  905.89KiB in 00:00:00 at 4.02MiB/s                  
[download] Destination: UCBR8-60-B28hp2BmDPdntcQ/DcNI16MEjrM.f140.m4a
[download] 100% of  568.33KiB in 00:00:00 at 20.37MiB/s  
[Merger] Merging formats into "UCBR8-60-B28hp2BmD

In [15]:
#生成XML表头模块
def xml_rss(title,link,description,category,icon,items):
    # 获取当前时间
    current_time_now = time.time()  # 获取当前时间的秒数
    # 获取当前时区和夏令时信息
    time_info_now = time.localtime(current_time_now)
    # 构造时间字符串
    formatted_time_now = time.strftime('%a, %d %b %Y %H:%M:%S %z', time_info_now)
    # 创建主XML信息
    return f'''<?xml version="1.0" encoding="UTF-8"?>
<rss version="2.0" xmlns:itunes="http://www.itunes.com/dtds/podcast-1.0.dtd">
    <channel>
        <title>{title}</title>
        <link>{link}</link>
        <description>{description}</description>
        <category>{category}</category>
        <generator>Podflow (support us at https://github.com/gruel-zxz/podflow)</generator>
        <language>en-us</language>
        <lastBuildDate>{formatted_time_now}</lastBuildDate>
        <pubDate>Sun, 24 Apr 2005 11:20:54 +0800</pubDate>
        <image>
            <url>{icon}</url>
            <title>{title}</title>
            <link>{link}</link>
        </image>
        <itunes:author>{title}</itunes:author>
        <itunes:subtitle>{title}</itunes:subtitle>
        <itunes:summary><![CDATA[{description}]]></itunes:summary>
        <itunes:image href="{icon}"></itunes:image>
        <itunes:explicit>no</itunes:explicit>
        <itunes:category text="{category}"></itunes:category>
{items}
    </channel>
</rss>'''

In [71]:
# 生成item模块
def xml_item(video_url, output_dir, video_website, channelid_title,title, description, pubDate, image):
    # 查看标题中是否有频道名称，如无添加到描述中
    if channelid_title not in html.unescape("title"):
        description = f"『{html.escape(channelid_title)}』\n{description}"
    # 更换描述换行符
    replacement_description = description.replace("\n", "&#xA;")
    # 获取文件后缀和文件字节大小
    if os.path.exists(f"{output_dir}/{video_url}.mp4"):
        video_length_bytes = os.path.getsize(f"{output_dir}/{video_url}.mp4")
        output_format = "mp4"
        video_type = "video/mp4"
    else:
        if os.path.exists(f"{output_dir}/{video_url}.m4a"):
            video_length_bytes = os.path.getsize(f"{output_dir}/{video_url}.m4a")
        else:
            video_length_bytes = 0
        output_format = "m4a"
        video_type = "audio/x-m4a"
    # 获取文件时长
    hours, remaining_seconds = divmod(get_duration_ffprobe("UCghLs6s95LrBWOdlZUCH4qw/lVbyYz3YW-o.m4a"), 3600)
    minutes = remaining_seconds // 60
    remaining_seconds = remaining_seconds % 60
    if hours > 1:
        duration = '{:02}:{:02}:{:02}'.format(hours, minutes, remaining_seconds)
    else:
        duration = '{:02}:{:02}'.format(minutes, remaining_seconds)
    # 回显对应的item
    return f'''
        <item>
            <guid>{video_url}</guid>
            <title>{title}</title>
            <link>{video_website}{video_url}</link>
            <description>{replacement_description}</description>
            <pubDate>{pubDate}</pubDate>
            <enclosure url="{config["url"]}/{output_dir}/{video_url}.{output_format}" length="{video_length_bytes}" type="{video_type}"></enclosure>
            <itunes:author>{title}</itunes:author>
            <itunes:subtitle>{title}</itunes:subtitle>
            <itunes:summary><![CDATA[{description}]]></itunes:summary>
            <itunes:image href="{image}"></itunes:image>
            <itunes:duration>{duration}</itunes:duration>
            <itunes:explicit>no</itunes:explicit>
            <itunes:order>1</itunes:order>
        </item>
'''

In [69]:
# 生成YouTube的item模块
def youtube_xml_item(entry):
    # 输入时间字符串和原始时区
    time_str = re.search(r"(?<=<published>).+(?=</published>)", entry).group()
    original_tz = timezone.utc  # 原始时区为UTC
    # 解析时间字符串并转换为datetime对象
    dt = datetime.fromisoformat(time_str[:-6]).replace(tzinfo=original_tz)
    # 转换为目标时区
    target_tz = timezone(timedelta(seconds=-(time.timezone + time.daylight)))
    dt_target = dt.astimezone(target_tz)
    # 格式化为目标时间字符串
    target_format = '%a, %d %b %Y %H:%M:%S %z'
    pubDate = dt_target.strftime(target_format)
    output_dir = re.search(r"(?<=<yt:channelId>).+(?=</yt:channelId>)", entry).group()
    description = re.search(r"(?<=<media:description>).+(?=</media:description>)", re.sub(r"\n+", "\n", entry), flags=re.DOTALL)
    description = description.group() if description else ""
    return xml_item(
        re.search(r"(?<=<yt:videoId>).+(?=</yt:videoId>)", entry).group(),
        output_dir ,
        "https://youtube.com/watch?v=",
        channelid_youtube[channelid_youtube_ids[output_dir ]]["title"],
        re.search(r"(?<=<title>).+(?=</title>)", entry).group(),
        description,
        pubDate,
        re.search(r"(?<=<media:thumbnail url=\").+(?=\" width=\")", entry).group()
    )


In [72]:
# 生成YouTube对应channel的需更新的items模块
def youtube_xml_items(output_dir):
    items = f"<!-- {output_dir} -->"
    with open(f"channel_id/{output_dir}.txt", 'r', encoding='utf-8') as file:  # 打开文件进行读取
        for entry in re.findall(r"<entry>.+?</entry>", file.read(), re.DOTALL)[:channelid_youtube[channelid_youtube_ids[output_dir ]]["update_size"]]:
            items = f"{items}{youtube_xml_item(entry)}<!-- {output_dir} -->"
    return items

In [73]:
print(youtube_xml_items("UCBR8-60-B28hp2BmDPdntcQ"))

<!-- UCBR8-60-B28hp2BmDPdntcQ -->
        <item>
            <guid>DcNI16MEjrM</guid>
            <title>seek real content and thou shall find 🤠 @OlimaOmega</title>
            <link>https://youtube.com/watch?v=DcNI16MEjrM</link>
            <description>『YouTube』&#xA;</description>
            <pubDate>Mon, 04 Sep 2023 18:00:16 +0000</pubDate>
            <enclosure url="http://127.0.0.1:8000/UCBR8-60-B28hp2BmDPdntcQ/DcNI16MEjrM.mp4" length="1525973" type="video/mp4"></enclosure>
            <itunes:author>seek real content and thou shall find 🤠 @OlimaOmega</itunes:author>
            <itunes:subtitle>seek real content and thou shall find 🤠 @OlimaOmega</itunes:subtitle>
            <itunes:summary><![CDATA[『YouTube』
]]></itunes:summary>
            <itunes:image href="https://i1.ytimg.com/vi/DcNI16MEjrM/hqdefault.jpg"></itunes:image>
            <itunes:duration>10:54</itunes:duration>
            <itunes:explicit>no</itunes:explicit>
            <itunes:order>1</itunes:order>
       

In [38]:

print(
    re.sub(
        r"=s(0|[1-9]\d{0,3}|1[0-9]{1,3}|20[0-3][0-9]|204[0-8])-c-k",
        "=s2048-c-k",
        re.search(
            r"https?://yt3.googleusercontent.com/[^\s]*(?=\">)",
            requests.get(
                "https://www.youtube.com/channel/UCBR8-60-B28hp2BmDPdntcQ/about"
            ).text,
        ).group(),
    )
)

https://yt3.googleusercontent.com/584JjRp5QMuKbyduM_2k5RlXFqHJtQ0qLIPZpwbUjMJmgzZngHcam5JMuZQxyzGMV5ljwJRl0Q=s2048-c-k-c0x00ffffff-no-rj


In [18]:
print(youtube_xml_item(''' <entry>
  <id>yt:video:lVbyYz3YW-o</id>
  <yt:videoId>lVbyYz3YW-o</yt:videoId>
  <yt:channelId>UCghLs6s95LrBWOdlZUCH4qw</yt:channelId>
  <title>北卡中国留学生把导师给就地正法了（20230829第6280期）</title>
  <link rel="alternate" href="https://www.youtube.com/watch?v=lVbyYz3YW-o"/>
  <author>
   <name>stone记</name>
   <uri>https://www.youtube.com/channel/UCghLs6s95LrBWOdlZUCH4qw</uri>
  </author>
  <published>2023-08-29T21:30:08+00:00</published>
  <updated>2023-09-01T01:06:29+00:00</updated>
  <media:group>
   <media:title>北卡中国留学生把导师给就地正法了（20230829第6280期）</media:title>
   <media:content url="https://www.youtube.com/v/lVbyYz3YW-o?version=3" type="application/x-shockwave-flash" width="640" height="390"/>
   <media:thumbnail url="https://i1.ytimg.com/vi/lVbyYz3YW-o/hqdefault.jpg" width="480" height="360"/>
   <media:description>加入stone记会员链接
https://www.youtube.com/channel/UCghLs6s95LrBWOdlZUCH4qw/join
合作联系Twitter：stone记@stone62855987
我们现阶段录制视频使用的设备
主力相机Canon EOS R
https://amzn.to/2L2eHQG
主力镜头Canon RF 50mm f/1.2L USM Lens
https://amzn.to/2L3qWwt
主力无线麦克风Sennheiser AVX 配合MKE2
https://amzn.to/2ZliYnj
桌面三角架
https://amzn.to/321gZqc
将两个麦克风合一的Saramonic SR-AX100
https://amzn.to/2TWRofg
铁威马D8 332
https://amzn.to/3BqlGxT

一些实用的附件推荐
相机背带Peak Design Slide Lite Camera Strap SLL-BK-3 (Black)
https://amzn.to/2QC0csh
相机腕带Peak Design Cuff Camera Wrist Strap
https://amzn.to/2S6Z4cZ
相机包Lowepro Adventura SH 160 II
https://amzn.to/2USOA2Q
三星T5 2TB移动硬盘
https://amzn.to/2rK72xf
佳能照片打印机Canon SELPHY CP1300
https://amzn.to/2EA2CBp
Flash Hot Shoe Mount Adapter热靴
https://amzn.to/2x9NPXU
Logitech BRIO
https://amzn.to/2Qp4Q9E
Rode VideoMicro 手机麦克风
https://amzn.to/2CLwquJ
JOBY GorillaPod 3K Kit 八爪魚三腳架
https://amzn.to/2QqAofl
Logitech G903
https://amzn.to/2QocOzv
Logitech G Powerplay Wireless Charging System
https://amzn.to/2x29EIQ
Blue Yeti USB Microphone 桌面麦克风
https://amzn.to/2x52Eet
Marathon CL030060WH 显示温度和湿度的桌面时钟
https://amzn.to/2CLyfYB
Marathon CL030054WH 显示温度和湿度的床头时钟
https://amzn.to/2QkOS02

介绍我们最近升级的新设备和心得（20181218第444期）
https://youtu.be/4G8yWQyBKak
介绍我们录制视频使用的全部设备（20180912第257期）
https://youtu.be/BbtTwsP_-zc</media:description>
   <media:community>
    <media:starRating count="393" average="5.00" min="1" max="5"/>
    <media:statistics views="87978"/>
   </media:community>
  </media:group>
 </entry>'''))

        <item>
            <guid>lVbyYz3YW-o</guid>
            <title>北卡中国留学生把导师给就地正法了（20230829第6280期）</title>
            <link>https://youtube.com/watch?v=lVbyYz3YW-o</link>
            <description>『stone记』&#xA;加入stone记会员链接&#xA;https://www.youtube.com/channel/UCghLs6s95LrBWOdlZUCH4qw/join&#xA;合作联系Twitter：stone记@stone62855987&#xA;我们现阶段录制视频使用的设备&#xA;主力相机Canon EOS R&#xA;https://amzn.to/2L2eHQG&#xA;主力镜头Canon RF 50mm f/1.2L USM Lens&#xA;https://amzn.to/2L3qWwt&#xA;主力无线麦克风Sennheiser AVX 配合MKE2&#xA;https://amzn.to/2ZliYnj&#xA;桌面三角架&#xA;https://amzn.to/321gZqc&#xA;将两个麦克风合一的Saramonic SR-AX100&#xA;https://amzn.to/2TWRofg&#xA;铁威马D8 332&#xA;https://amzn.to/3BqlGxT&#xA;一些实用的附件推荐&#xA;相机背带Peak Design Slide Lite Camera Strap SLL-BK-3 (Black)&#xA;https://amzn.to/2QC0csh&#xA;相机腕带Peak Design Cuff Camera Wrist Strap&#xA;https://amzn.to/2S6Z4cZ&#xA;相机包Lowepro Adventura SH 160 II&#xA;https://amzn.to/2USOA2Q&#xA;三星T5 2TB移动硬盘&#xA;https://amzn.to/2rK72xf&#xA;佳能照片打印机Canon SELPHY CP1300&#xA;https://amzn.to/2

In [19]:
print(xml_rss(config["title"], config["link"], config["description"], config["category"], config["icon"], ""))
print(config["url"])

<?xml version="1.0" encoding="UTF-8"?>
<rss version="2.0" xmlns:itunes="http://www.itunes.com/dtds/podcast-1.0.dtd">
    <channel>
        <title>YouTube</title>
        <link>https://m.youtube.com</link>
        <description>在YouTube 上畅享您喜爱的视频和音乐，上传原创内容并与亲朋好友和全世界观众分享您的视频。</description>
        <category>TV &amp; Film</category>
        <generator>Podflow (support us at https://github.com/gruel-zxz/podflow)</generator>
        <language>en-us</language>
        <lastBuildDate>Tue, 05 Sep 2023 00:31:10 +0000</lastBuildDate>
        <pubDate>Sun, 24 Apr 2005 11:20:54 +0800</pubDate>
        <image>
            <url>https://static.vecteezy.com/system/resources/previews/003/399/771/original/youtube-icon-editorial-free-vector.jpg</url>
            <title>YouTube</title>
            <link>https://m.youtube.com</link>
        </image>
        <itunes:author>YouTube</itunes:author>
        <itunes:subtitle>YouTube</itunes:subtitle>
        <itunes:summary><![CDATA[在YouTube 上畅享您喜爱的视频和音乐，上传原创内容并